# Sozialauswahl pro Kopf

In [19]:
import csv
import datetime


class Mitarbeiter:
    def __init__(self, vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert):
        self.vorname = vorname
        self.nachname = nachname
        self.alter = alter
        self.betriebszugehoerigkeit = betriebszugehoerigkeit
        self.ehepartner_unterhalt = ehepartner_unterhalt
        self.kinder_unterhalt = kinder_unterhalt
        self.schwerbehindert = schwerbehindert
        self.punkte = alter + 1.5 * betriebszugehoerigkeit + 5 * ehepartner_unterhalt + 5 * kinder_unterhalt + 8 * schwerbehindert
        self.dismiss = False
        self.gruppe = None
        
# Für den Fall, dass das Geburtsdatum und nicht das Alter in den Daten steht        
def berechne_alter(geburtsdatum):
    heute = datetime.date.today()  # Heutiges Datum
    geburtsdatum = datetime.datetime.strptime(geburtsdatum, '%Y-%m-%d').date()  # Geburtsdatum in Date-Format umwandeln
    alter = heute.year - geburtsdatum.year - ((heute.month, heute.day) < (geburtsdatum.month, geburtsdatum.day))
    return alter

def read_data(filename):
    mitarbeiter_list = []
    with open(filename, 'r', newline='') as file:
        reader = csv.reader(file, delimiter=',') # Ggf Delimiter anpassen
        fieldnames = next(reader)
        fieldnames = [field.strip().lower() for field in fieldnames]
        reader = csv.DictReader(file, fieldnames=fieldnames)
        
        for row in reader:
            vorname = row['vorname'].strip()
            nachname = row['nachname'].strip()
            name = f"{vorname} {nachname}"
            
            # Berechnen des Alters basierend auf dem Geburtsdatum oder direkt verwenden, falls vorhanden
            if 'geburtsdatum' in row and row['geburtsdatum'].strip():
                geburtsdatum = row['geburtsdatum'].strip()
                alter = berechne_alter(geburtsdatum)
            elif 'alter' in row and row['alter'].strip():
                alter = int(row['alter'].strip())
            else:
                raise ValueError("Weder Alter noch Geburtsdatum angegeben")
            
            betriebszugehoerigkeit = int(row['betriebszugehoerigkeit'].strip())
            ehepartner_unterhalt = int(row['ehepartnerunterhalt'].strip())
            kinder_unterhalt = int(row['kinderunterhalt'].strip())
            schwerbehindert = int(row['schwerbehindert'].strip())
            
            mitarbeiter = Mitarbeiter(vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert)
            mitarbeiter_list.append(mitarbeiter)
            
    return mitarbeiter_list

# Definition der Gruppen 
def assign_groups(mitarbeiter_list):
    groups = {1: [], 2: [], 3: [], 4: []}
    for mitarbeiter in mitarbeiter_list:
        if mitarbeiter.alter <= 30:
            group = 1
        elif 31 <= mitarbeiter.alter <= 40:
            group = 2
        elif 41 <= mitarbeiter.alter <= 50:
            group = 3
        else:
            group = 4
#Einsortieren der Mitarbeiter
        mitarbeiter.gruppe = group
        groups[group].append(mitarbeiter)
    return groups

# Entlassungslogik
def mark_for_dismissal(groups, total_dismissals):
    # Ermittlung der Anzahl der Entlassungsempfehlungen pro Gruppe
    dismissals_per_group = total_dismissals // 4
    remaining_dismissals = total_dismissals - dismissals_per_group * 4 
    # Den Schlüssel (Gruppennummer) und den Wert (Mitarbeiter) dem Dictionary zuweisen
    for group, mitarbeiter_list in groups.items():
        sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: x.punkte)
        actual_dismissals = min(dismissals_per_group, len(sorted_mitarbeiter))
        for i in range(actual_dismissals):
            sorted_mitarbeiter[i].dismiss = True
   # Verteilen der restlichen Entlassungsempfehlungen nach Punkten
    remaining_mitarbeiter = [mitarbeiter for sublist in groups.values() for mitarbeiter in sublist if not mitarbeiter.dismiss]
    sorted_remaining_mitarbeiter = sorted(remaining_mitarbeiter, key=lambda x: x.punkte)
    for i in range(min(remaining_dismissals, len(sorted_remaining_mitarbeiter))):
        sorted_remaining_mitarbeiter[i].dismiss = True
        
  #Kontrollausgabe
def print_results(groups):
    for group, mitarbeiter_list in groups.items():
        #Ausgabe nach Gruppen und Sortierung nach Punkten in aufsteigender Reihenfolge
        sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: (-x.dismiss, -x.punkte))
        print(f"Gruppe {group}:")
        for mitarbeiter in sorted_mitarbeiter:
            if mitarbeiter.dismiss:
                print(f"{mitarbeiter.vorname} {mitarbeiter.nachname} (Punkte: {mitarbeiter.punkte}) - Entlassung empfohlen")
            else:
                print(f"{mitarbeiter.vorname} {mitarbeiter.nachname} (Punkte: {mitarbeiter.punkte})")



#Speichern der Ergebnisse in eine CSV-Datei
def save_results_to_csv(groups, filename='results.csv'): 
                               # Der Dateiname ist ein Standardwert und kann im main-Teil geändert  werden.
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Gruppe', 'Vorname', 'Nachname', 'Alter', 'Betriebszugehoerigkeit', 'Ehepartner_Unterhalt', 
                      'Kinder_Unterhalt', 'schwerbehindert', 'Punkte', 'Empfehlung']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for group, mitarbeiter_list in sorted(groups.items()):
            sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x:(-x.dismiss, -x.punkte))
            for mitarbeiter in sorted_mitarbeiter:
                writer.writerow({
                    'Gruppe': group,
                    'Vorname': mitarbeiter.vorname,
                    'Nachname': mitarbeiter.nachname,
                    'Alter': mitarbeiter.alter,
                    'Betriebszugehoerigkeit': mitarbeiter.betriebszugehoerigkeit,
                    'Ehepartner_Unterhalt': 'ja' if mitarbeiter.ehepartner_unterhalt == 1
                                                 else '',
                    'Kinder_Unterhalt': mitarbeiter.kinder_unterhalt,
                    'schwerbehindert': 'ja' if mitarbeiter.schwerbehindert == 1 
                                            else '',
                    'Punkte': mitarbeiter.punkte,
                    'Empfehlung': 'Entlassungsempfehlung' if mitarbeiter.dismiss 
                                       else ''
                })


def main():
    filename = 'new_names_synthetic_employees.csv' # Die Ausgangsdatei anpassen
    total_dismissals = 10
    mitarbeiter_list = read_data(filename)
    if mitarbeiter_list:
        groups = assign_groups(mitarbeiter_list)
        mark_for_dismissal(groups, total_dismissals)
        print_results(groups)
        save_results_to_csv(groups, 'Entlassungsempfehlung_FTE.csv') # Passenden Dateinamen wählen

if __name__ == "__main__":
    main()


Gruppe 1:
Megan Marshall (Punkte: 43.5) - Entlassung empfohlen
Robert Martin (Punkte: 36.5) - Entlassung empfohlen
Emily Pearson (Punkte: 35.5) - Entlassung empfohlen
Paul Taylor (Punkte: 34.5) - Entlassung empfohlen
Michelle Hernandez (Punkte: 62.0)
Zachary Mitchell (Punkte: 57.5)
Cindy Parker (Punkte: 55.0)
Garrett Vasquez (Punkte: 53.5)
George Moyer (Punkte: 52.5)
Katelyn Snyder (Punkte: 46.5)
Leslie Martinez (Punkte: 45.5)
Daniel Gallegos (Punkte: 45.0)
Tara Burgess (Punkte: 44.5)
Gruppe 2:
Robin Skinner (Punkte: 54.5) - Entlassung empfohlen
Gregory Page (Punkte: 48.5) - Entlassung empfohlen
Erik Good (Punkte: 85.0)
Monica King (Punkte: 79.0)
Debra Myers (Punkte: 71.0)
Gruppe 3:
Andrea Johnson (Punkte: 62.5) - Entlassung empfohlen
Tamara Richards (Punkte: 57.5) - Entlassung empfohlen
Alex Alexander (Punkte: 103.5)
Amy Nichols (Punkte: 101.5)
Tyler Ponce (Punkte: 99.5)
Susan Thompson (Punkte: 90.0)
Alvin Rollins (Punkte: 84.5)
Stephen Price (Punkte: 84.5)
Gabriel Wilkins (Punkte: 77